In [81]:
##Author: Hayden Gill
##Purpose: This section takes the comments and posts, and reformats them for classification of price movement

import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
#from datetime import strftime
import re
import swifter
import math
from multiprocessing import Pool

In [2]:
comments = pd.read_csv("comments_with_ids_topics.csv")
posts = pd.read_csv("posts_with_ids_topics.csv")


In [3]:
display(comments)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,post_id,comment_id
0,0,0.413798,0.305130,0.012508,0.012504,0.012504,0.012505,0.193533,0.012508,0.012504,0.012507,t3_kcvkwx,gftdfnp
1,1,0.774962,0.025004,0.025004,0.025003,0.025003,0.025003,0.025010,0.025003,0.025003,0.025003,t3_kcvkwx,gftdfk8
2,2,0.020016,0.340247,0.251509,0.020020,0.020015,0.268127,0.020015,0.020017,0.020015,0.020020,t1_gftd073,gftdfh0
3,3,0.033343,0.033338,0.033341,0.033336,0.033336,0.033363,0.033336,0.699931,0.033341,0.033336,t3_kcvkwx,gftdfek
4,4,0.033340,0.366545,0.033340,0.033340,0.366728,0.033340,0.033345,0.033340,0.033340,0.033342,t3_kcvkwx,gftdfb7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217254,5217254,0.016703,0.288875,0.181828,0.016702,0.016702,0.016703,0.016706,0.412375,0.016704,0.016702,t1_gkdmaa1,gkdmwie
5217255,5217255,0.360699,0.012537,0.012545,0.012540,0.012535,0.261118,0.012535,0.290420,0.012535,0.012537,t1_gkceomw,gkdmw91
5217256,5217256,0.012512,0.012512,0.012518,0.887385,0.012512,0.012512,0.012514,0.012512,0.012512,0.012513,t3_l3c0hj,gkdmw6k
5217257,5217257,0.194306,0.000000,0.307896,0.000000,0.000000,0.365310,0.000000,0.000000,0.099102,0.000000,t1_gkbyl29,gkdmvxa


In [4]:
display(posts)

,Unnamed: 0,0,1,2,3,4,id
0,0,0.011203,0.011199,0.649899,0.316192,0.011506,kcz0hf
1,1,0.179181,0.000000,0.607012,0.209607,0.000000,kcyyo1
2,2,0.000000,0.000000,0.303152,0.678872,0.000000,kcyxbk
3,3,0.013800,0.013550,0.583235,0.375656,0.013759,kcywij
4,4,0.015556,0.015506,0.015573,0.937628,0.015737,kcyk2w
...,...,...,...,...,...,...,...
53422,53422,0.012782,0.013102,0.801328,0.159845,0.012944,kl8wxo
53423,53423,0.128956,0.000000,0.859260,0.000000,0.000000,kl8wbp
53424,53424,0.051568,0.315584,0.051588,0.529828,0.051432,kl8m4y
53425,53425,0.033696,0.034024,0.033827,0.864023,0.034429,kl8f6l


In [5]:
display(pd.read_csv("wsb_comments_results.csv"))

,Unnamed: 0,author,body,created_utc,id,parent_id,subreddit,subreddit_id,body_tickers
0,0,guesswhatihate,dipping,1607957482,gftdfol,t1_gftdcb8,wallstreetbets,t5_2th52,NaN
1,1,k12nmonky,nio and spce ftw,1607957482,gftdfo4,t1_gftdc1s,wallstreetbets,t5_2th52,NaN
2,2,tnmtnmtnm,flat,1607957482,gftdfo3,t1_gftdbth,wallstreetbets,t5_2th52,NaN
3,3,everlastingdeath,lol what happened to the limit up everyones be...,1607957481,gftdfnp,t3_kcvkwx,wallstreetbets,t5_2th52,NaN
4,4,ZakkuTurner,jan,1607957481,gftdfmf,t1_gftddng,wallstreetbets,t5_2th52,NaN
...,...,...,...,...,...,...,...,...,...
9559642,9559652,AngryBobRoss,lmaogot nights to sleep over itmight belittle ...,1611412578,gkdmw91,t1_gkceomw,wallstreetbets,t5_2th52,NaN
9559643,9559653,Machin_Shins,squeeze aside are there price targets for gmes...,1611412578,gkdmw6k,t3_l3c0hj,wallstreetbets,t5_2th52,NaN
9559644,9559654,testfire10,td ameritrade,1611412576,gkdmw1y,t1_gkdloa8,wallstreetbets,t5_2th52,NaN
9559645,9559655,stupidlysweet,same ive gotin savings but thinking of droppin...,1611412575,gkdmvxa,t1_gkbyl29,wallstreetbets,t5_2th52,NaN


In [6]:
display(pd.read_csv("wsb_post_results.csv"))

,Unnamed: 0,author,author_premium,created_utc,domain,id,num_comments,body,subreddit,subreddit_id,title,upvote_ratio,body_tickers,title_tickers
0,0,readyrummy1,False,1607957430,self.wallstreetbets,kcz0hf,6,am woke up five minutes ago made some instant...,wallstreetbets,t5_2th52,ruled by the theta gods,1.0,NaN,NaN
1,1,Pluto_Muto,False,1607957404,self.wallstreetbets,kcz07r,0,removed,wallstreetbets,t5_2th52,k andwant more,1.0,NaN,NaN
2,3,uslashuname,False,1607957282,self.wallstreetbets,kcyyw4,0,removed,wallstreetbets,t5_2th52,azn fair value of price of vaccine coming so...,1.0,NaN,NaN
3,4,dadwhomstfucks12,False,1607957280,self.wallstreetbets,kcyyve,0,removed,wallstreetbets,t5_2th52,i wanna be less retarded,1.0,NaN,NaN
4,5,officespace2020,False,1607957260,self.wallstreetbets,kcyyo1,9,intro let me get soft with ya it can get ugly...,wallstreetbets,t5_2th52,motivation monday,1.0,LRCX,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424102,707451,Salahforisis,False,1609094763,self.wallstreetbets,kl8wbp,337,i dont know how the fuck you guys became so po...,wallstreetbets,t5_2th52,you fucking knuckleheads got my mom to reinves...,1.0,NaN,NaN
424103,707452,Decent-Cress,False,1609094505,self.wallstreetbets,kl8tjy,0,removed,wallstreetbets,t5_2th52,immp whathomerun short for utopia chatroom,1.0,NaN,NaN
424104,707454,Junkbot,False,1609093826,self.wallstreetbets,kl8m4y,4,saw the trends and tryingback test to see if h...,wallstreetbets,t5_2th52,gme gold,1.0,NaN,NaN
424105,707459,ZookeepergameSoggy49,False,1609093189,self.wallstreetbets,kl8f6l,18,large gamestop investor ryan cohen buys more s...,wallstreetbets,t5_2th52,was itdream gme opens on monday up,1.0,NaN,NaN


In [7]:
comments_ids_dates = pd.read_csv("wsb_comments_results.csv").loc[:,['created_utc', 'id']]
display(comments_ids_dates)

,created_utc,id
0,1607957482,gftdfol
1,1607957482,gftdfo4
2,1607957482,gftdfo3
3,1607957481,gftdfnp
4,1607957481,gftdfmf
...,...,...
9559642,1611412578,gkdmw91
9559643,1611412578,gkdmw6k
9559644,1611412576,gkdmw1y
9559645,1611412575,gkdmvxa


In [8]:
posts_ids_dates = pd.read_csv("wsb_post_results.csv").loc[:,['created_utc','id']]
display(posts_ids_dates)

,created_utc,id
0,1607957430,kcz0hf
1,1607957404,kcz07r
2,1607957282,kcyyw4
3,1607957280,kcyyve
4,1607957260,kcyyo1
...,...,...
424102,1609094763,kl8wbp
424103,1609094505,kl8tjy
424104,1609093826,kl8m4y
424105,1609093189,kl8f6l


In [9]:
##create aggregate-able dates 
posts_dates = np.array([datetime.fromtimestamp(date) for date in posts_ids_dates['created_utc']])
mask = np.array([date.hour < 9 for date in posts_dates])
posts_dates[mask] = posts_dates[mask] + timedelta(days = -1)
posts_dates = [date.strftime("%m/%d/%Y") for date in posts_dates]
posts_ids_dates['date'] = posts_dates
posts_ids_dates = posts_ids_dates.iloc[:,1:]
display(posts_ids_dates)
#display(posts_dates)

comments_dates = np.array([datetime.fromtimestamp(date) for date in comments_ids_dates['created_utc']])
mask = np.array([date.hour < 9 for date in comments_dates])
comments_dates[mask] = comments_dates[mask] + timedelta(days = -1)
comments_dates = [date.strftime("%m/%d/%Y") for date in comments_dates]
comments_ids_dates['date'] = comments_dates
comments_ids_dates = comments_ids_dates.iloc[:,1:]
display(comments_ids_dates)

,id,date
0,kcz0hf,12/13/2020
1,kcz07r,12/13/2020
2,kcyyw4,12/13/2020
3,kcyyve,12/13/2020
4,kcyyo1,12/13/2020
...,...,...
424102,kl8wbp,12/27/2020
424103,kl8tjy,12/27/2020
424104,kl8m4y,12/27/2020
424105,kl8f6l,12/27/2020


,id,date
0,gftdfol,12/13/2020
1,gftdfo4,12/13/2020
2,gftdfo3,12/13/2020
3,gftdfnp,12/13/2020
4,gftdfmf,12/13/2020
...,...,...
9559642,gkdmw91,01/22/2021
9559643,gkdmw6k,01/22/2021
9559644,gkdmw1y,01/22/2021
9559645,gkdmvxa,01/22/2021


NOTE: AT THIS POINT WE ARE PREPARING FOR MODELING AS DESCRIBED - THE DATA HAS BEEN INCREMENTED SO THAT <9AM IS "YESTERDAY" - AND THE DATES NO LONGER CORRESPOND WITH "TRUTH"

In [10]:
comments['date'] = comments_ids_dates['date']
display(comments)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,post_id,comment_id,date
0,0,0.413798,0.305130,0.012508,0.012504,0.012504,0.012505,0.193533,0.012508,0.012504,0.012507,t3_kcvkwx,gftdfnp,12/13/2020
1,1,0.774962,0.025004,0.025004,0.025003,0.025003,0.025003,0.025010,0.025003,0.025003,0.025003,t3_kcvkwx,gftdfk8,12/13/2020
2,2,0.020016,0.340247,0.251509,0.020020,0.020015,0.268127,0.020015,0.020017,0.020015,0.020020,t1_gftd073,gftdfh0,12/13/2020
3,3,0.033343,0.033338,0.033341,0.033336,0.033336,0.033363,0.033336,0.699931,0.033341,0.033336,t3_kcvkwx,gftdfek,12/13/2020
4,4,0.033340,0.366545,0.033340,0.033340,0.366728,0.033340,0.033345,0.033340,0.033340,0.033342,t3_kcvkwx,gftdfb7,12/13/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217254,5217254,0.016703,0.288875,0.181828,0.016702,0.016702,0.016703,0.016706,0.412375,0.016704,0.016702,t1_gkdmaa1,gkdmwie,01/29/2021
5217255,5217255,0.360699,0.012537,0.012545,0.012540,0.012535,0.261118,0.012535,0.290420,0.012535,0.012537,t1_gkceomw,gkdmw91,01/29/2021
5217256,5217256,0.012512,0.012512,0.012518,0.887385,0.012512,0.012512,0.012514,0.012512,0.012512,0.012513,t3_l3c0hj,gkdmw6k,01/29/2021
5217257,5217257,0.194306,0.000000,0.307896,0.000000,0.000000,0.365310,0.000000,0.000000,0.099102,0.000000,t1_gkbyl29,gkdmvxa,01/29/2021


In [11]:
posts['date'] = posts_ids_dates['date']
display(posts)

,Unnamed: 0,0,1,2,3,4,id,date
0,0,0.011203,0.011199,0.649899,0.316192,0.011506,kcz0hf,12/13/2020
1,1,0.179181,0.000000,0.607012,0.209607,0.000000,kcyyo1,12/13/2020
2,2,0.000000,0.000000,0.303152,0.678872,0.000000,kcyxbk,12/13/2020
3,3,0.013800,0.013550,0.583235,0.375656,0.013759,kcywij,12/13/2020
4,4,0.015556,0.015506,0.015573,0.937628,0.015737,kcyk2w,12/13/2020
...,...,...,...,...,...,...,...,...
53422,53422,0.012782,0.013102,0.801328,0.159845,0.012944,kl8wxo,01/31/2021
53423,53423,0.128956,0.000000,0.859260,0.000000,0.000000,kl8wbp,01/31/2021
53424,53424,0.051568,0.315584,0.051588,0.529828,0.051432,kl8m4y,01/31/2021
53425,53425,0.033696,0.034024,0.033827,0.864023,0.034429,kl8f6l,01/31/2021


In [12]:
comment_sentiment_results = pd.read_csv("Comments_with_vader_sentiment.csv")
post_sentiment_results = pd.read_csv("Posts_with_vader_sentiment.csv")

In [13]:
display(comment_sentiment_results)

,Unnamed: 0,neg,neu,pos,compound,id,parent_id
0,0,0.000,0.859,0.141,0.4215,gftdfnp,t3_kcvkwx
1,1,0.000,1.000,0.000,0.0000,gftdfk8,t3_kcvkwx
2,2,0.257,0.495,0.248,-0.0258,gftdfh0,t1_gftd073
3,3,0.000,0.541,0.459,0.6423,gftdfek,t3_kcvkwx
4,4,0.000,1.000,0.000,0.0000,gftdfb7,t3_kcvkwx
...,...,...,...,...,...,...,...
5217254,5217254,0.259,0.647,0.094,-0.5106,gkdmwie,t1_gkdmaa1
5217255,5217255,0.195,0.805,0.000,-0.4404,gkdmw91,t1_gkceomw
5217256,5217256,0.000,1.000,0.000,0.0000,gkdmw6k,t3_l3c0hj
5217257,5217257,0.282,0.636,0.082,-0.8599,gkdmvxa,t1_gkbyl29


In [14]:
display(post_sentiment_results)

,Unnamed: 0,neg,neu,pos,compound,id
0,0,0.136,0.803,0.062,-0.9212,kcz0hf
1,1,0.129,0.739,0.131,-0.7838,kcyyo1
2,2,0.000,0.847,0.153,0.9313,kcyxbk
3,3,0.191,0.739,0.071,-0.8226,kcywij
4,4,0.000,0.919,0.081,0.8834,kcyk2w
...,...,...,...,...,...,...
53422,53422,0.152,0.728,0.119,-0.3612,kl8wxo
53423,53423,0.115,0.812,0.073,-0.9142,kl8wbp
53424,53424,0.000,0.823,0.177,0.4215,kl8m4y
53425,53425,0.000,1.000,0.000,0.0000,kl8f6l


In [15]:
print(type(post_sentiment_results['id']))
print(type(posts['id']))
posts.id.astype(str)
post_sentiment_results.astype(str)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,Unnamed: 0,neg,neu,pos,compound,id
0,0,0.136,0.8029999999999999,0.062,-0.9212,kcz0hf
1,1,0.129,0.7390000000000001,0.131,-0.7838,kcyyo1
2,2,0.0,0.847,0.153,0.9313,kcyxbk
3,3,0.191,0.7390000000000001,0.071,-0.8226,kcywij
4,4,0.0,0.919,0.081,0.8834,kcyk2w
...,...,...,...,...,...,...
53422,53422,0.152,0.728,0.11900000000000001,-0.3612,kl8wxo
53423,53423,0.115,0.812,0.073,-0.9142,kl8wbp
53424,53424,0.0,0.823,0.177,0.4215,kl8m4y
53425,53425,0.0,1.0,0.0,0.0,kl8f6l


In [16]:
posts['neg'] = post_sentiment_results['neg']
posts['pos'] = post_sentiment_results['pos']
posts['neu'] = post_sentiment_results['neu']
posts['compound'] = post_sentiment_results['compound']
display(posts)

,Unnamed: 0,0,1,2,3,4,id,date,neg,pos,neu,compound
0,0,0.011203,0.011199,0.649899,0.316192,0.011506,kcz0hf,12/13/2020,0.136,0.062,0.803,-0.9212
1,1,0.179181,0.000000,0.607012,0.209607,0.000000,kcyyo1,12/13/2020,0.129,0.131,0.739,-0.7838
2,2,0.000000,0.000000,0.303152,0.678872,0.000000,kcyxbk,12/13/2020,0.000,0.153,0.847,0.9313
3,3,0.013800,0.013550,0.583235,0.375656,0.013759,kcywij,12/13/2020,0.191,0.071,0.739,-0.8226
4,4,0.015556,0.015506,0.015573,0.937628,0.015737,kcyk2w,12/13/2020,0.000,0.081,0.919,0.8834
...,...,...,...,...,...,...,...,...,...,...,...,...
53422,53422,0.012782,0.013102,0.801328,0.159845,0.012944,kl8wxo,01/31/2021,0.152,0.119,0.728,-0.3612
53423,53423,0.128956,0.000000,0.859260,0.000000,0.000000,kl8wbp,01/31/2021,0.115,0.073,0.812,-0.9142
53424,53424,0.051568,0.315584,0.051588,0.529828,0.051432,kl8m4y,01/31/2021,0.000,0.177,0.823,0.4215
53425,53425,0.033696,0.034024,0.033827,0.864023,0.034429,kl8f6l,01/31/2021,0.000,0.000,1.000,0.0000


In [17]:
comments['neg'] = comment_sentiment_results['neg']
comments['pos'] = comment_sentiment_results['pos']
comments['neu'] = comment_sentiment_results['neu']
comments['compound'] = comment_sentiment_results['compound']

In [18]:
temp = comments.copy(deep=True)
temp.columns = [str("comment_" + currentname) for currentname in comments.columns]
display(temp)

,comment_Unnamed: 0,comment_0,comment_1,comment_2,comment_3,comment_4,comment_5,comment_6,comment_7,comment_8,comment_9,comment_post_id,comment_comment_id,comment_date,comment_neg,comment_pos,comment_neu,comment_compound
0,0,0.413798,0.305130,0.012508,0.012504,0.012504,0.012505,0.193533,0.012508,0.012504,0.012507,t3_kcvkwx,gftdfnp,12/13/2020,0.000,0.141,0.859,0.4215
1,1,0.774962,0.025004,0.025004,0.025003,0.025003,0.025003,0.025010,0.025003,0.025003,0.025003,t3_kcvkwx,gftdfk8,12/13/2020,0.000,0.000,1.000,0.0000
2,2,0.020016,0.340247,0.251509,0.020020,0.020015,0.268127,0.020015,0.020017,0.020015,0.020020,t1_gftd073,gftdfh0,12/13/2020,0.257,0.248,0.495,-0.0258
3,3,0.033343,0.033338,0.033341,0.033336,0.033336,0.033363,0.033336,0.699931,0.033341,0.033336,t3_kcvkwx,gftdfek,12/13/2020,0.000,0.459,0.541,0.6423
4,4,0.033340,0.366545,0.033340,0.033340,0.366728,0.033340,0.033345,0.033340,0.033340,0.033342,t3_kcvkwx,gftdfb7,12/13/2020,0.000,0.000,1.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217254,5217254,0.016703,0.288875,0.181828,0.016702,0.016702,0.016703,0.016706,0.412375,0.016704,0.016702,t1_gkdmaa1,gkdmwie,01/29/2021,0.259,0.094,0.647,-0.5106
5217255,5217255,0.360699,0.012537,0.012545,0.012540,0.012535,0.261118,0.012535,0.290420,0.012535,0.012537,t1_gkceomw,gkdmw91,01/29/2021,0.195,0.000,0.805,-0.4404
5217256,5217256,0.012512,0.012512,0.012518,0.887385,0.012512,0.012512,0.012514,0.012512,0.012512,0.012513,t3_l3c0hj,gkdmw6k,01/29/2021,0.000,0.000,1.000,0.0000
5217257,5217257,0.194306,0.000000,0.307896,0.000000,0.000000,0.365310,0.000000,0.000000,0.099102,0.000000,t1_gkbyl29,gkdmvxa,01/29/2021,0.282,0.082,0.636,-0.8599


NOW - need to aggregate before we unify - since will unify by dates - first save work

In [41]:
comments.to_csv("finalized_comments.csv")
posts.to_csv("finalized_posts.csv")

In [19]:
comments = pd.read_csv("finalized_comments.csv")
posts = pd.read_csv("finalized_posts.csv")

In [20]:
display(posts)

,Unnamed: 0,Unnamed: 0.1,0,1,2,3,4,id,date,neg,pos,neu,compound
0,0,0,0.011203,0.011199,0.649899,0.316192,0.011506,kcz0hf,12/13/2020,0.136,0.062,0.803,-0.9212
1,1,1,0.179181,0.000000,0.607012,0.209607,0.000000,kcyyo1,12/13/2020,0.129,0.131,0.739,-0.7838
2,2,2,0.000000,0.000000,0.303152,0.678872,0.000000,kcyxbk,12/13/2020,0.000,0.153,0.847,0.9313
3,3,3,0.013800,0.013550,0.583235,0.375656,0.013759,kcywij,12/13/2020,0.191,0.071,0.739,-0.8226
4,4,4,0.015556,0.015506,0.015573,0.937628,0.015737,kcyk2w,12/13/2020,0.000,0.081,0.919,0.8834
...,...,...,...,...,...,...,...,...,...,...,...,...,...
53422,53422,53422,0.012782,0.013102,0.801328,0.159845,0.012944,kl8wxo,01/31/2021,0.152,0.119,0.728,-0.3612
53423,53423,53423,0.128956,0.000000,0.859260,0.000000,0.000000,kl8wbp,01/31/2021,0.115,0.073,0.812,-0.9142
53424,53424,53424,0.051568,0.315584,0.051588,0.529828,0.051432,kl8m4y,01/31/2021,0.000,0.177,0.823,0.4215
53425,53425,53425,0.033696,0.034024,0.033827,0.864023,0.034429,kl8f6l,01/31/2021,0.000,0.000,1.000,0.0000


In [21]:
temp = posts.iloc[:,2:]
temp.columns = ["(post)Short_Squeeze","(post)Buy_More!","(post)My_Commitment","(post)Tactics","(post)The_Enemy", "post_id","post_date","(post)neg_sent","(post)pos_sent","(post)neu_sent","(post)compound_sent"] 
display(temp)
posts = temp

,(post)Short_Squeeze,(post)Buy_More!,(post)My_Commitment,(post)Tactics,(post)The_Enemy,post_id,post_date,(post)neg_sent,(post)pos_sent,(post)neu_sent,(post)compound_sent
0,0.011203,0.011199,0.649899,0.316192,0.011506,kcz0hf,12/13/2020,0.136,0.062,0.803,-0.9212
1,0.179181,0.000000,0.607012,0.209607,0.000000,kcyyo1,12/13/2020,0.129,0.131,0.739,-0.7838
2,0.000000,0.000000,0.303152,0.678872,0.000000,kcyxbk,12/13/2020,0.000,0.153,0.847,0.9313
3,0.013800,0.013550,0.583235,0.375656,0.013759,kcywij,12/13/2020,0.191,0.071,0.739,-0.8226
4,0.015556,0.015506,0.015573,0.937628,0.015737,kcyk2w,12/13/2020,0.000,0.081,0.919,0.8834
...,...,...,...,...,...,...,...,...,...,...,...
53422,0.012782,0.013102,0.801328,0.159845,0.012944,kl8wxo,01/31/2021,0.152,0.119,0.728,-0.3612
53423,0.128956,0.000000,0.859260,0.000000,0.000000,kl8wbp,01/31/2021,0.115,0.073,0.812,-0.9142
53424,0.051568,0.315584,0.051588,0.529828,0.051432,kl8m4y,01/31/2021,0.000,0.177,0.823,0.4215
53425,0.033696,0.034024,0.033827,0.864023,0.034429,kl8f6l,01/31/2021,0.000,0.000,1.000,0.0000


In [22]:
temp = comments.iloc[:,2:]
temp.columns = ["(com)Determination","(com)Frustration","(com)Gains","(com)Shorting","(com)GME/AMC","(com)Uncertainty","(com)Tactics","(com)Shittalking","(com)Technology","(com)The_Enemy","post_id","comment_id","(com)neg_sent","(com)pos_sent","(com)neu_sent","(com)compound_sent"]
comments = temp
display(comments)

,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,post_id,comment_id,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent
0,0.413798,0.305130,0.012508,0.012504,0.012504,0.012505,0.193533,0.012508,0.012504,0.012507,t3_kcvkwx,gftdfnp,0.000,0.141,0.859,0.4215
1,0.774962,0.025004,0.025004,0.025003,0.025003,0.025003,0.025010,0.025003,0.025003,0.025003,t3_kcvkwx,gftdfk8,0.000,0.000,1.000,0.0000
2,0.020016,0.340247,0.251509,0.020020,0.020015,0.268127,0.020015,0.020017,0.020015,0.020020,t1_gftd073,gftdfh0,0.257,0.248,0.495,-0.0258
3,0.033343,0.033338,0.033341,0.033336,0.033336,0.033363,0.033336,0.699931,0.033341,0.033336,t3_kcvkwx,gftdfek,0.000,0.459,0.541,0.6423
4,0.033340,0.366545,0.033340,0.033340,0.366728,0.033340,0.033345,0.033340,0.033340,0.033342,t3_kcvkwx,gftdfb7,0.000,0.000,1.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217254,0.016703,0.288875,0.181828,0.016702,0.016702,0.016703,0.016706,0.412375,0.016704,0.016702,t1_gkdmaa1,gkdmwie,0.259,0.094,0.647,-0.5106
5217255,0.360699,0.012537,0.012545,0.012540,0.012535,0.261118,0.012535,0.290420,0.012535,0.012537,t1_gkceomw,gkdmw91,0.195,0.000,0.805,-0.4404
5217256,0.012512,0.012512,0.012518,0.887385,0.012512,0.012512,0.012514,0.012512,0.012512,0.012513,t3_l3c0hj,gkdmw6k,0.000,0.000,1.000,0.0000
5217257,0.194306,0.000000,0.307896,0.000000,0.000000,0.365310,0.000000,0.000000,0.099102,0.000000,t1_gkbyl29,gkdmvxa,0.282,0.082,0.636,-0.8599


In [23]:
##remove any comments other than t_1 to reduce the noise
#all_t1 = ["t1_.*" in postid for postid in comments['post_id']]
t1_mask = [bool(re.search('^t1_.*', postid)) for postid in comments['post_id']]
print(sum(t1_mask))
t1_comments = comments.iloc[t1_mask,:]
display(t1_comments)

2817928


,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,post_id,comment_id,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent
2,0.020016,0.340247,0.251509,0.020020,0.020015,0.268127,0.020015,0.020017,0.020015,0.020020,t1_gftd073,gftdfh0,0.257,0.248,0.495,-0.0258
6,0.000000,0.036516,0.114062,0.000000,0.000000,0.155298,0.170482,0.317614,0.193888,0.000000,t1_gftcijz,gftdf4z,0.090,0.000,0.910,-0.7783
11,0.025021,0.025010,0.025022,0.025012,0.025011,0.025014,0.025008,0.774877,0.025010,0.025014,t1_gffzkfq,gftdest,0.000,0.000,1.000,0.0000
12,0.000000,0.000000,0.000000,0.552381,0.000000,0.000000,0.000000,0.124244,0.000000,0.302144,t1_gftc4c3,gftdepw,0.114,0.151,0.736,0.2100
13,0.014301,0.311923,0.014293,0.014296,0.014297,0.014293,0.573719,0.014292,0.014293,0.014292,t1_gftdbdo,gftdepu,0.000,0.000,1.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5217252,0.000000,0.000000,0.000000,0.000000,0.000000,0.132642,0.000000,0.000000,0.000000,0.800570,t1_gkb9hsw,gkdmwu8,0.000,0.290,0.710,0.6486
5217254,0.016703,0.288875,0.181828,0.016702,0.016702,0.016703,0.016706,0.412375,0.016704,0.016702,t1_gkdmaa1,gkdmwie,0.259,0.094,0.647,-0.5106
5217255,0.360699,0.012537,0.012545,0.012540,0.012535,0.261118,0.012535,0.290420,0.012535,0.012537,t1_gkceomw,gkdmw91,0.195,0.000,0.805,-0.4404
5217257,0.194306,0.000000,0.307896,0.000000,0.000000,0.365310,0.000000,0.000000,0.099102,0.000000,t1_gkbyl29,gkdmvxa,0.282,0.082,0.636,-0.8599


In [70]:
print(t1_comments.post_id.value_counts()[:10])

t1_gl5a2mt    499
t1_gl34srm    474
t1_glah4ov    462
t1_gl35xpy    393
t1_glheepo    332
t1_gls5wj7    299
t1_glqidn6    299
t1_glqsnjy    261
t1_gle13f8    257
t1_gl4xfvh    251
Name: post_id, dtype: int64


In [26]:
def summarize_comments(df):
    #comment_id = df.iloc[0,11]
    df = df.drop(labels = ['post_id','comment_id'], axis = 1)
    comment_base = len(df)
    summary = df.mean(axis=0)
    #summary['comment_base'] = comment_base
    return summary
small_sample = t1_comments.iloc[:1000,:]
display(small_sample.groupby('post_id').apply(lambda x: summarize_comments(x)))

,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent
post_id,,,,,,,,,,,,,,
t1_gd4nghn,0.000000,0.000000,0.000000,0.696124,0.000000,0.259391,0.000000,0.000000,0.000000,0.000000,0.000,0.062,0.938,0.3182
t1_geot5g9,0.000000,0.000000,0.000000,0.918155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.186,0.814,0.4939
t1_gf7nrc6,0.620155,0.020016,0.020020,0.020019,0.020018,0.020027,0.020020,0.020017,0.219688,0.020020,0.000,0.426,0.574,0.6369
t1_gfdhu51,0.050007,0.050007,0.050007,0.050017,0.050007,0.050010,0.050007,0.050007,0.050007,0.549921,0.259,0.271,0.471,0.0258
t1_gfdrdbn,0.016693,0.016693,0.439081,0.016696,0.016692,0.427366,0.016692,0.016694,0.016694,0.016700,0.367,0.327,0.306,-0.1027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t1_glwab5g,0.012523,0.012520,0.204927,0.283175,0.012518,0.284082,0.012518,0.152698,0.012520,0.012518,0.166,0.000,0.834,-0.3774
t1_glwab7l,0.050009,0.050021,0.050009,0.050009,0.050009,0.050009,0.050009,0.549894,0.050016,0.050017,0.000,0.000,1.000,0.0000
t1_glwabgy,0.399700,0.262627,0.000000,0.000000,0.000000,0.000000,0.179664,0.000000,0.000000,0.115123,0.123,0.000,0.877,-0.2732


In [28]:
with Pool(7) as p:
    groupby = t1_comments.groupby('post_id')
    print("grouped")
    groups = [groupby.get_group(group) for group in groupby.groups]
    print("listed, pooling")
    summarized_comments = p.map(summarize_comments, groups)
display(summarized_comments)

grouped


KeyboardInterrupt: 

Once this is done -we will have posts combined with comments. We need to just aggregate up to the level of days and attempt a prediction - this is quasi-second order - but not a true second order - interaction

In [25]:
summarized_comments = comments.groupby("post_id").apply(lambda x: summarize_comments(x))
display(summarized_comments)

,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent
post_id,,,,,,,,,,,,,,
t1_fvygh1t,0.000000,0.000000,0.269625,0.000000,0.000000,0.130120,0.000000,0.000000,0.099959,0.445677,0.127000,0.219000,0.6540,0.340000
t1_fwjns5m,0.020012,0.220120,0.020017,0.020014,0.020012,0.020014,0.020012,0.419676,0.220112,0.020012,0.000000,0.385000,0.6150,0.361200
t1_fxbq8je,0.016683,0.192731,0.016683,0.016684,0.016682,0.016687,0.016682,0.016683,0.673802,0.016683,0.000000,0.000000,1.0000,0.000000
t1_fxh8uo6,0.020025,0.020028,0.020030,0.020032,0.020025,0.020028,0.246802,0.219672,0.193623,0.219734,0.000000,0.221000,0.7790,0.372400
t1_fxu63r2,0.774882,0.025012,0.025012,0.025012,0.025016,0.025012,0.025012,0.025017,0.025012,0.025013,0.000000,0.000000,1.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t3_lbvg19,0.120295,0.005562,0.155397,0.135559,0.076627,0.127579,0.005562,0.035176,0.119872,0.203720,0.104000,0.234333,0.6610,0.510617
t3_lbvgl7,0.490314,0.197753,0.020852,0.020851,0.082704,0.020854,0.020851,0.020855,0.020850,0.104115,0.000000,0.246000,0.7540,0.396500
t3_lbvhhv,0.106783,0.131333,0.051331,0.221914,0.028670,0.177013,0.015274,0.098957,0.115269,0.047715,0.056067,0.171733,0.7722,0.182227


In [32]:
##Some T_2+ got through again, will remove now
t1_mask = [bool(re.search('^t1_.*', postid)) for postid in summarized_comments.index]
print(sum(t1_mask))
summarized_t1_comments = summarized_comments.iloc[t1_mask,:]

1635227


In [34]:
display(summarized_t1_comments)

,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent
post_id,,,,,,,,,,,,,,
t1_fvygh1t,0.000000,0.000000,0.269625,0.000000,0.000000,0.130120,0.000000,0.000000,0.099959,0.445677,0.127,0.219,0.654,0.3400
t1_fwjns5m,0.020012,0.220120,0.020017,0.020014,0.020012,0.020014,0.020012,0.419676,0.220112,0.020012,0.000,0.385,0.615,0.3612
t1_fxbq8je,0.016683,0.192731,0.016683,0.016684,0.016682,0.016687,0.016682,0.016683,0.673802,0.016683,0.000,0.000,1.000,0.0000
t1_fxh8uo6,0.020025,0.020028,0.020030,0.020032,0.020025,0.020028,0.246802,0.219672,0.193623,0.219734,0.000,0.221,0.779,0.3724
t1_fxu63r2,0.774882,0.025012,0.025012,0.025012,0.025016,0.025012,0.025012,0.025017,0.025012,0.025013,0.000,0.000,1.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t1_glwab5g,0.012523,0.012520,0.205009,0.283213,0.012518,0.283956,0.012518,0.152705,0.012520,0.012518,0.166,0.000,0.834,-0.3774
t1_glwab7l,0.050009,0.050021,0.050009,0.050009,0.050009,0.050009,0.050009,0.549894,0.050016,0.050017,0.000,0.000,1.000,0.0000
t1_glwabgy,0.399694,0.262629,0.000000,0.000000,0.000000,0.000000,0.179651,0.000000,0.000000,0.115139,0.123,0.000,0.877,-0.2732


In [36]:
temp = summarized_t1_comments
temp.index = [re.sub("^t1_", "", postid) for postid in temp.index]
display(temp)
summarized_t1_comments = temp

,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent
fvygh1t,0.000000,0.000000,0.269625,0.000000,0.000000,0.130120,0.000000,0.000000,0.099959,0.445677,0.127,0.219,0.654,0.3400
fwjns5m,0.020012,0.220120,0.020017,0.020014,0.020012,0.020014,0.020012,0.419676,0.220112,0.020012,0.000,0.385,0.615,0.3612
fxbq8je,0.016683,0.192731,0.016683,0.016684,0.016682,0.016687,0.016682,0.016683,0.673802,0.016683,0.000,0.000,1.000,0.0000
fxh8uo6,0.020025,0.020028,0.020030,0.020032,0.020025,0.020028,0.246802,0.219672,0.193623,0.219734,0.000,0.221,0.779,0.3724
fxu63r2,0.774882,0.025012,0.025012,0.025012,0.025016,0.025012,0.025012,0.025017,0.025012,0.025013,0.000,0.000,1.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
glwab5g,0.012523,0.012520,0.205009,0.283213,0.012518,0.283956,0.012518,0.152705,0.012520,0.012518,0.166,0.000,0.834,-0.3774
glwab7l,0.050009,0.050021,0.050009,0.050009,0.050009,0.050009,0.050009,0.549894,0.050016,0.050017,0.000,0.000,1.000,0.0000
glwabgy,0.399694,0.262629,0.000000,0.000000,0.000000,0.000000,0.179651,0.000000,0.000000,0.115139,0.123,0.000,0.877,-0.2732
glwaeca,0.020005,0.020008,0.311794,0.020005,0.020003,0.272931,0.020005,0.275237,0.020004,0.020007,0.000,0.223,0.777,0.3182


In [57]:
##last-minute issues popping up - instead of adding comments to posts, add dates to comments and aggregate by date to combine in
temp = [re.sub("^t._","",postid) for postid in summarized_comments.index]
#tempindex = [range(0,len(summarized_comments), 1)]
temp_summarized = summarized_comments.copy(deep=True)
temp_summarized['post_id'] = temp
temp_summarized.index = range(len(summarized_comments))
posts_ids_dates_temp = posts_ids_dates.copy(deep = True)
posts_ids_dates_temp.columns = ['post_id', 'date']

In [69]:
temp_summarized['post_id'] = temp_summarized['post_id'].astype("str")
display(temp_summarized)

,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent,post_id
0,0.000000,0.000000,0.269625,0.000000,0.000000,0.130120,0.000000,0.000000,0.099959,0.445677,0.127000,0.219000,0.6540,0.340000,fvygh1t
1,0.020012,0.220120,0.020017,0.020014,0.020012,0.020014,0.020012,0.419676,0.220112,0.020012,0.000000,0.385000,0.6150,0.361200,fwjns5m
2,0.016683,0.192731,0.016683,0.016684,0.016682,0.016687,0.016682,0.016683,0.673802,0.016683,0.000000,0.000000,1.0000,0.000000,fxbq8je
3,0.020025,0.020028,0.020030,0.020032,0.020025,0.020028,0.246802,0.219672,0.193623,0.219734,0.000000,0.221000,0.7790,0.372400,fxh8uo6
4,0.774882,0.025012,0.025012,0.025012,0.025016,0.025012,0.025012,0.025017,0.025012,0.025013,0.000000,0.000000,1.0000,0.000000,fxu63r2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713881,0.120295,0.005562,0.155397,0.135559,0.076627,0.127579,0.005562,0.035176,0.119872,0.203720,0.104000,0.234333,0.6610,0.510617,lbvg19
1713882,0.490314,0.197753,0.020852,0.020851,0.082704,0.020854,0.020851,0.020855,0.020850,0.104115,0.000000,0.246000,0.7540,0.396500,lbvgl7
1713883,0.106783,0.131333,0.051331,0.221914,0.028670,0.177013,0.015274,0.098957,0.115269,0.047715,0.056067,0.171733,0.7722,0.182227,lbvhhv
1713884,0.014312,0.014321,0.014312,0.014314,0.014312,0.014312,0.282503,0.014312,0.344901,0.272399,0.000000,0.000000,1.0000,0.000000,lbvjff


In [66]:
posts_ids_dates_temp['post_id'] = posts_ids_dates_temp['post_id'].astype("str")
posts_ids_dates_temp.dtypes
display(posts_ids_dates_temp)

,post_id,date
0,kcz0hf,12/13/2020
1,kcz07r,12/13/2020
2,kcyyw4,12/13/2020
3,kcyyve,12/13/2020
4,kcyyo1,12/13/2020
...,...,...
424102,kl8wbp,12/27/2020
424103,kl8tjy,12/27/2020
424104,kl8m4y,12/27/2020
424105,kl8f6l,12/27/2020


In [71]:
temp_summarized['post_id']

0          fvygh1t
1          fwjns5m
2          fxbq8je
3          fxh8uo6
4          fxu63r2
            ...   
1713881     lbvg19
1713882     lbvgl7
1713883     lbvhhv
1713884     lbvjff
1713885     lbvjzl
Name: post_id, Length: 1713886, dtype: object

In [72]:
temp_summarized = temp_summarized.join(posts_ids_dates)
display(temp_summarized)

,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent,post_id,id,date
0,0.000000,0.000000,0.269625,0.000000,0.000000,0.130120,0.000000,0.000000,0.099959,0.445677,0.127000,0.219000,0.6540,0.340000,fvygh1t,kcz0hf,12/13/2020
1,0.020012,0.220120,0.020017,0.020014,0.020012,0.020014,0.020012,0.419676,0.220112,0.020012,0.000000,0.385000,0.6150,0.361200,fwjns5m,kcz07r,12/13/2020
2,0.016683,0.192731,0.016683,0.016684,0.016682,0.016687,0.016682,0.016683,0.673802,0.016683,0.000000,0.000000,1.0000,0.000000,fxbq8je,kcyyw4,12/13/2020
3,0.020025,0.020028,0.020030,0.020032,0.020025,0.020028,0.246802,0.219672,0.193623,0.219734,0.000000,0.221000,0.7790,0.372400,fxh8uo6,kcyyve,12/13/2020
4,0.774882,0.025012,0.025012,0.025012,0.025016,0.025012,0.025012,0.025017,0.025012,0.025013,0.000000,0.000000,1.0000,0.000000,fxu63r2,kcyyo1,12/13/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713881,0.120295,0.005562,0.155397,0.135559,0.076627,0.127579,0.005562,0.035176,0.119872,0.203720,0.104000,0.234333,0.6610,0.510617,lbvg19,NaN,NaN
1713882,0.490314,0.197753,0.020852,0.020851,0.082704,0.020854,0.020851,0.020855,0.020850,0.104115,0.000000,0.246000,0.7540,0.396500,lbvgl7,NaN,NaN
1713883,0.106783,0.131333,0.051331,0.221914,0.028670,0.177013,0.015274,0.098957,0.115269,0.047715,0.056067,0.171733,0.7722,0.182227,lbvhhv,NaN,NaN
1713884,0.014312,0.014321,0.014312,0.014314,0.014312,0.014312,0.282503,0.014312,0.344901,0.272399,0.000000,0.000000,1.0000,0.000000,lbvjff,NaN,NaN


In [79]:
temp_summarized['date'] = temp_summarized['date'].astype(str)
temp_summarized['id'] = temp_summarized['id'].astype(str)

In [89]:
sum([str(item) == str('NaN') for item in temp_summarized['id']])
##outrageous. beyond my comprehension why this is not working at the worst possible time

0

In [91]:
temp_summarized.to_csv("summarized_comment_topics_sentiments_withdate.csv")

In [98]:
temp_temp_summarized = pd.read_csv("summarized_comment_topics_sentiments_bydate.csv", dtype={'post_id':str, 'id':str,'date':str})

In [124]:
display(temp_temp_summarized)

,Unnamed: 0,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent,post_id,id,date
0,0,0.000000,0.000000,0.269625,0.000000,0.000000,0.130120,0.000000,0.000000,0.099959,0.445677,0.127000,0.219000,0.6540,0.340000,fvygh1t,kcz0hf,12/13/2020
1,1,0.020012,0.220120,0.020017,0.020014,0.020012,0.020014,0.020012,0.419676,0.220112,0.020012,0.000000,0.385000,0.6150,0.361200,fwjns5m,kcz07r,12/13/2020
2,2,0.016683,0.192731,0.016683,0.016684,0.016682,0.016687,0.016682,0.016683,0.673802,0.016683,0.000000,0.000000,1.0000,0.000000,fxbq8je,kcyyw4,12/13/2020
3,3,0.020025,0.020028,0.020030,0.020032,0.020025,0.020028,0.246802,0.219672,0.193623,0.219734,0.000000,0.221000,0.7790,0.372400,fxh8uo6,kcyyve,12/13/2020
4,4,0.774882,0.025012,0.025012,0.025012,0.025016,0.025012,0.025012,0.025017,0.025012,0.025013,0.000000,0.000000,1.0000,0.000000,fxu63r2,kcyyo1,12/13/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1713881,1713881,0.120295,0.005562,0.155397,0.135559,0.076627,0.127579,0.005562,0.035176,0.119872,0.203720,0.104000,0.234333,0.6610,0.510617,lbvg19,NaN,NaN
1713882,1713882,0.490314,0.197753,0.020852,0.020851,0.082704,0.020854,0.020851,0.020855,0.020850,0.104115,0.000000,0.246000,0.7540,0.396500,lbvgl7,NaN,NaN
1713883,1713883,0.106783,0.131333,0.051331,0.221914,0.028670,0.177013,0.015274,0.098957,0.115269,0.047715,0.056067,0.171733,0.7722,0.182227,lbvhhv,NaN,NaN
1713884,1713884,0.014312,0.014321,0.014312,0.014314,0.014312,0.014312,0.282503,0.014312,0.344901,0.272399,0.000000,0.000000,1.0000,0.000000,lbvjff,NaN,NaN


In [129]:
mask = [isinstance(item, str) for item in temp_temp_summarized.iloc[:,17]]
sum(mask)
##after the 424108th item, all become int nans. must remove them
temp_temp_summarized = temp_temp_summarized.iloc[mask,:]

In [130]:
display(temp_temp_summarized)

,Unnamed: 0,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent,post_id,id,date
0,0,0.000000,0.000000,0.269625,0.000000,0.000000,0.130120,0.000000,0.000000,0.099959,0.445677,0.127000,0.219000,0.654,0.340000,fvygh1t,kcz0hf,12/13/2020
1,1,0.020012,0.220120,0.020017,0.020014,0.020012,0.020014,0.020012,0.419676,0.220112,0.020012,0.000000,0.385000,0.615,0.361200,fwjns5m,kcz07r,12/13/2020
2,2,0.016683,0.192731,0.016683,0.016684,0.016682,0.016687,0.016682,0.016683,0.673802,0.016683,0.000000,0.000000,1.000,0.000000,fxbq8je,kcyyw4,12/13/2020
3,3,0.020025,0.020028,0.020030,0.020032,0.020025,0.020028,0.246802,0.219672,0.193623,0.219734,0.000000,0.221000,0.779,0.372400,fxh8uo6,kcyyve,12/13/2020
4,4,0.774882,0.025012,0.025012,0.025012,0.025016,0.025012,0.025012,0.025017,0.025012,0.025013,0.000000,0.000000,1.000,0.000000,fxu63r2,kcyyo1,12/13/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424102,424102,0.025005,0.025003,0.025003,0.025003,0.025002,0.025005,0.025002,0.025005,0.774969,0.025003,0.000000,0.000000,1.000,0.000000,gi9r8w6,kl8wbp,12/27/2020
424103,424103,0.146988,0.000000,0.097762,0.424898,0.070546,0.000000,0.237019,0.000000,0.000000,0.000000,0.000000,0.322000,0.678,0.928700,gi9r8y0,kl8tjy,12/27/2020
424104,424104,0.025015,0.025005,0.025007,0.025003,0.025004,0.774950,0.025002,0.025006,0.025006,0.025003,0.000000,0.000000,1.000,0.000000,gi9r90z,kl8m4y,12/27/2020
424105,424105,0.000000,0.000000,0.421563,0.000000,0.107613,0.000000,0.300999,0.000000,0.000000,0.115166,0.062000,0.152000,0.787,0.296000,gi9r9ar,kl8f6l,12/27/2020


In [132]:
def summarize_comments_again(df):
    #comment_id = df.iloc[0,11]
    df = df.drop(labels = ['post_id','id'], axis = 1)
    comment_base = len(df)
    summary = df.mean(axis=0)
    #summary['comment_base'] = comment_base
    return summary
comments_daily_summary = temp_temp_summarized.groupby('date').apply(lambda x: summarize_comments_again(x))
display(comments_daily_summary)

,Unnamed: 0,(com)Determination,(com)Frustration,(com)Gains,(com)Shorting,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent
date,,,,,,,,,,,,,,,
01/01/2021,14167.264069,0.120947,0.083869,0.111018,0.186836,0.041086,0.117860,0.087498,0.107489,0.055469,0.076323,0.081330,0.117436,0.801253,0.054275
01/02/2021,61852.883895,0.109643,0.084500,0.120881,0.174635,0.053087,0.110471,0.072587,0.132667,0.046247,0.083452,0.096730,0.108312,0.794971,0.041088
01/03/2021,102089.173021,0.123524,0.089835,0.168791,0.142992,0.044896,0.102550,0.064537,0.120385,0.053367,0.079780,0.083730,0.137419,0.778857,0.093897
01/04/2021,20961.696897,0.117359,0.090970,0.118542,0.179421,0.045246,0.095817,0.100611,0.120480,0.045667,0.075457,0.081972,0.121286,0.796743,0.087688
01/05/2021,52983.944844,0.126971,0.073285,0.131626,0.150496,0.055704,0.108406,0.078510,0.125788,0.059750,0.079025,0.073402,0.123836,0.802760,0.092297
01/06/2021,95236.989080,0.125578,0.094913,0.129120,0.144961,0.039745,0.107675,0.088098,0.128346,0.048446,0.082630,0.083974,0.131313,0.784715,0.097103
01/07/2021,19884.472266,0.101316,0.084955,0.121062,0.176789,0.035613,0.115012,0.095489,0.132251,0.054817,0.071370,0.091376,0.112228,0.796399,0.037350
01/08/2021,41328.318519,0.112400,0.081121,0.131877,0.161625,0.046654,0.110059,0.084086,0.135499,0.044492,0.082277,0.090347,0.111915,0.797743,0.058061
01/09/2021,95931.515000,0.111315,0.079568,0.138306,0.164602,0.040448,0.107389,0.079131,0.124033,0.060412,0.085622,0.086201,0.116074,0.797726,0.065287


In [133]:
display(posts)

,(post)Short_Squeeze,(post)Buy_More!,(post)My_Commitment,(post)Tactics,(post)The_Enemy,post_id,post_date,(post)neg_sent,(post)pos_sent,(post)neu_sent,(post)compound_sent
0,0.011203,0.011199,0.649899,0.316192,0.011506,kcz0hf,12/13/2020,0.136,0.062,0.803,-0.9212
1,0.179181,0.000000,0.607012,0.209607,0.000000,kcyyo1,12/13/2020,0.129,0.131,0.739,-0.7838
2,0.000000,0.000000,0.303152,0.678872,0.000000,kcyxbk,12/13/2020,0.000,0.153,0.847,0.9313
3,0.013800,0.013550,0.583235,0.375656,0.013759,kcywij,12/13/2020,0.191,0.071,0.739,-0.8226
4,0.015556,0.015506,0.015573,0.937628,0.015737,kcyk2w,12/13/2020,0.000,0.081,0.919,0.8834
...,...,...,...,...,...,...,...,...,...,...,...
53422,0.012782,0.013102,0.801328,0.159845,0.012944,kl8wxo,01/31/2021,0.152,0.119,0.728,-0.3612
53423,0.128956,0.000000,0.859260,0.000000,0.000000,kl8wbp,01/31/2021,0.115,0.073,0.812,-0.9142
53424,0.051568,0.315584,0.051588,0.529828,0.051432,kl8m4y,01/31/2021,0.000,0.177,0.823,0.4215
53425,0.033696,0.034024,0.033827,0.864023,0.034429,kl8f6l,01/31/2021,0.000,0.000,1.000,0.0000


In [134]:
def summarize_posts(df):
    #comment_id = df.iloc[0,11]
    df = df.drop(labels = ['post_id'], axis = 1)
    comment_base = len(df)
    summary = df.mean(axis=0)
    #summary['comment_base'] = comment_base
    return summary
posts_daily_summary = posts.groupby('post_date').apply(lambda x: summarize_posts(x))
display(posts_daily_summary)

,(post)Short_Squeeze,(post)Buy_More!,(post)My_Commitment,(post)Tactics,(post)The_Enemy,(post)neg_sent,(post)pos_sent,(post)neu_sent,(post)compound_sent
post_date,,,,,,,,,
01/01/2021,0.180975,0.097825,0.305699,0.335649,0.073988,0.064134,0.107000,0.828857,0.221239
01/02/2021,0.167146,0.082874,0.351054,0.340686,0.052429,0.068627,0.109541,0.821854,0.267774
01/03/2021,0.161383,0.142529,0.359631,0.254843,0.077166,0.074335,0.103444,0.822184,0.214811
01/04/2021,0.168268,0.240943,0.258493,0.212538,0.116469,0.069723,0.092704,0.837570,0.105950
01/05/2021,0.154709,0.163135,0.323664,0.217727,0.136524,0.086480,0.103008,0.810504,0.100369
01/06/2021,0.165621,0.166607,0.276534,0.220649,0.166079,0.079644,0.093890,0.826470,0.113663
01/07/2021,0.187364,0.172174,0.274800,0.230011,0.131183,0.074772,0.099674,0.825571,0.137222
01/08/2021,0.165398,0.137214,0.283712,0.287845,0.120455,0.067667,0.103253,0.829092,0.203639
01/09/2021,0.146787,0.081415,0.326603,0.350655,0.088733,0.062437,0.110525,0.827016,0.268306


In [142]:
##FINALLY - FINAL ASSEMBLY OF PREDICTORS - they all line up by date, so can just be added together
predictors = posts_daily_summary.join(comments_daily_summary)
display(predictors)

,(post)Short_Squeeze,(post)Buy_More!,(post)My_Commitment,(post)Tactics,(post)The_Enemy,(post)neg_sent,(post)pos_sent,(post)neu_sent,(post)compound_sent,Unnamed: 0,...,(com)GME/AMC,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent
post_date,,,,,,,,,,,,,,,,,,,,,
01/01/2021,0.180975,0.097825,0.305699,0.335649,0.073988,0.064134,0.107000,0.828857,0.221239,14167.264069,...,0.041086,0.117860,0.087498,0.107489,0.055469,0.076323,0.081330,0.117436,0.801253,0.054275
01/02/2021,0.167146,0.082874,0.351054,0.340686,0.052429,0.068627,0.109541,0.821854,0.267774,61852.883895,...,0.053087,0.110471,0.072587,0.132667,0.046247,0.083452,0.096730,0.108312,0.794971,0.041088
01/03/2021,0.161383,0.142529,0.359631,0.254843,0.077166,0.074335,0.103444,0.822184,0.214811,102089.173021,...,0.044896,0.102550,0.064537,0.120385,0.053367,0.079780,0.083730,0.137419,0.778857,0.093897
01/04/2021,0.168268,0.240943,0.258493,0.212538,0.116469,0.069723,0.092704,0.837570,0.105950,20961.696897,...,0.045246,0.095817,0.100611,0.120480,0.045667,0.075457,0.081972,0.121286,0.796743,0.087688
01/05/2021,0.154709,0.163135,0.323664,0.217727,0.136524,0.086480,0.103008,0.810504,0.100369,52983.944844,...,0.055704,0.108406,0.078510,0.125788,0.059750,0.079025,0.073402,0.123836,0.802760,0.092297
01/06/2021,0.165621,0.166607,0.276534,0.220649,0.166079,0.079644,0.093890,0.826470,0.113663,95236.989080,...,0.039745,0.107675,0.088098,0.128346,0.048446,0.082630,0.083974,0.131313,0.784715,0.097103
01/07/2021,0.187364,0.172174,0.274800,0.230011,0.131183,0.074772,0.099674,0.825571,0.137222,19884.472266,...,0.035613,0.115012,0.095489,0.132251,0.054817,0.071370,0.091376,0.112228,0.796399,0.037350
01/08/2021,0.165398,0.137214,0.283712,0.287845,0.120455,0.067667,0.103253,0.829092,0.203639,41328.318519,...,0.046654,0.110059,0.084086,0.135499,0.044492,0.082277,0.090347,0.111915,0.797743,0.058061
01/09/2021,0.146787,0.081415,0.326603,0.350655,0.088733,0.062437,0.110525,0.827016,0.268306,95931.515000,...,0.040448,0.107389,0.079131,0.124033,0.060412,0.085622,0.086201,0.116074,0.797726,0.065287


In [147]:
response = pd.read_csv("gme_movement.csv")

In [148]:
response = response.set_index('effective_date')
display(response)

,label
effective_date,
1/1/2021,Up
1/2/2021,Up
1/3/2021,Up
1/4/2021,Down
1/5/2021,Up
1/6/2021,Up
1/7/2021,Down
1/8/2021,Down
1/9/2021,Up


In [152]:
predictors['label'] = [label for label in response.label]
display(predictors)

,(post)Short_Squeeze,(post)Buy_More!,(post)My_Commitment,(post)Tactics,(post)The_Enemy,(post)neg_sent,(post)pos_sent,(post)neu_sent,(post)compound_sent,Unnamed: 0,...,(com)Uncertainty,(com)Tactics,(com)Shittalking,(com)Technology,(com)The_Enemy,(com)neg_sent,(com)pos_sent,(com)neu_sent,(com)compound_sent,label
post_date,,,,,,,,,,,,,,,,,,,,,
01/01/2021,0.180975,0.097825,0.305699,0.335649,0.073988,0.064134,0.107000,0.828857,0.221239,14167.264069,...,0.117860,0.087498,0.107489,0.055469,0.076323,0.081330,0.117436,0.801253,0.054275,Up
01/02/2021,0.167146,0.082874,0.351054,0.340686,0.052429,0.068627,0.109541,0.821854,0.267774,61852.883895,...,0.110471,0.072587,0.132667,0.046247,0.083452,0.096730,0.108312,0.794971,0.041088,Up
01/03/2021,0.161383,0.142529,0.359631,0.254843,0.077166,0.074335,0.103444,0.822184,0.214811,102089.173021,...,0.102550,0.064537,0.120385,0.053367,0.079780,0.083730,0.137419,0.778857,0.093897,Up
01/04/2021,0.168268,0.240943,0.258493,0.212538,0.116469,0.069723,0.092704,0.837570,0.105950,20961.696897,...,0.095817,0.100611,0.120480,0.045667,0.075457,0.081972,0.121286,0.796743,0.087688,Down
01/05/2021,0.154709,0.163135,0.323664,0.217727,0.136524,0.086480,0.103008,0.810504,0.100369,52983.944844,...,0.108406,0.078510,0.125788,0.059750,0.079025,0.073402,0.123836,0.802760,0.092297,Up
01/06/2021,0.165621,0.166607,0.276534,0.220649,0.166079,0.079644,0.093890,0.826470,0.113663,95236.989080,...,0.107675,0.088098,0.128346,0.048446,0.082630,0.083974,0.131313,0.784715,0.097103,Up
01/07/2021,0.187364,0.172174,0.274800,0.230011,0.131183,0.074772,0.099674,0.825571,0.137222,19884.472266,...,0.115012,0.095489,0.132251,0.054817,0.071370,0.091376,0.112228,0.796399,0.037350,Down
01/08/2021,0.165398,0.137214,0.283712,0.287845,0.120455,0.067667,0.103253,0.829092,0.203639,41328.318519,...,0.110059,0.084086,0.135499,0.044492,0.082277,0.090347,0.111915,0.797743,0.058061,Down
01/09/2021,0.146787,0.081415,0.326603,0.350655,0.088733,0.062437,0.110525,0.827016,0.268306,95931.515000,...,0.107389,0.079131,0.124033,0.060412,0.085622,0.086201,0.116074,0.797726,0.065287,Up


In [153]:
predictors.to_csv("experiment1.csv")